In [81]:
%load_ext watermark
%watermark -v -p numpy,scipy,sklearn,pandas,matplotlib

CPython 3.6.5
IPython 4.0.0

numpy 1.14.5
scipy 1.1.0
sklearn 0.19.2
pandas 0.23.3
matplotlib 2.2.2


In [82]:
# 파이썬 2와 파이썬 3 지원
from __future__ import division, print_function, unicode_literals

# 공통
import numpy as np
import os

# 일관된 출력을 위해 유사난수 초기화
np.random.seed(42)

# 맷플롯립 설정
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12


In [71]:
import tensorflow as tf
import pandas as pd

In [3]:
# MNIST 데이터 임포트
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [41]:
# 매개변수 정의
element_size = 28
time_steps = 28
num_classess = 10
batch_size =128
hidden_layer_size = 128

# 텐서보드 모델 요약을 저장할 위치
LOG_DIR = "logs/RNN_with_summaries"

# 입력과 레이블을 위한 플레이스홀더 생성
_inputs = tf.placeholder(tf.float32, shape=[None, time_steps, element_size], name='inputs')
y = tf.placeholder(tf.float32, shape=[None, num_classess], name='labels')


In [5]:
batch_x, batch_y = mnist.train.next_batch(batch_size)
# 28개의 시퀀스를 얻기 위해 각 데이터를 28필셀의 형태로 변환
batch_x = batch_x.reshape((batch_size, time_steps, element_size))

## RNN 단계

In [16]:
# 요약을 로깅하는 몇몇 연산을 추가하는 헬퍼 함수(텐서플로 문서에서 발췌)
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.scalar('histogram', var)
        

In [118]:
def rnn_step(previous_hidden_state, x):
    current_hidden_state = tf.tanh(tf.matmul(previous_hidden_state, Wh) + tf.matmul(x, Wx) + b_rnn)
    print('previous_hidden_state {}'.format(previous_hidden_state))
    print('Wh {}'.format(Wh))
    print('x {}'.format(x))
    print('Wx {}'.format(Wx))
    print('b_rnn {}'.format(b_rnn))
    hh = tf.matmul(previous_hidden_state, Wh)
    print('hh {}'.format(hh))
    xx = tf.matmul(x, Wx)
    print('xx {}'.format(xx))
    multi_1 = hh + xx
    print('multi_1 {}'.format(multi_1))
    
    return current_hidden_state

In [48]:
# Apply linear layer to state vector
def get_linear_layer(hidden_state):

    return tf.matmul(hidden_state, Wl) + bl

In [126]:
# 입력 및 은닉 계층에 적용할 가중치와 편향값
with tf.name_scope('rnn_weights'):
    with tf.name_scope("W_x"):
        Wx = tf.Variable(tf.zeros([element_size, hidden_layer_size]))
        variable_summaries(Wx)
    with tf.name_scope("W_h"):
        Wh = tf.Variable(tf.zeros([hidden_layer_size,hidden_layer_size]))
        variable_summaries(Wh)
    with tf.name_scope("Bias"):
        b_rnn = tf.Variable(tf.zeros([hidden_layer_size]))
        variable_summaries(b_rnn)

        

In [120]:
# scan 함수로 입력값 처리
# 입력의 형태 : (batch_size, time_steps, element_size)
processed_input = tf.transpose(_inputs, perm=[1,0,2])
# 변형된 입력의 형태 : (time_steps, batch_size, element_size)
initial_hidden = tf.zeros([batch_size, hidden_layer_size])
# 시간의 흐름에 따른 상태 백터 구하기
all_hidden_states = tf.scan(rnn_step,
                            processed_input, 
                           initializer=initial_hidden,
                           name='states')

previous_hidden_state Tensor("states_11/while/Identity_2:0", shape=(2, 128), dtype=float32)
Wh <tf.Variable 'rnn_weights_11/W_h/Variable:0' shape=(128, 128) dtype=float32_ref>
x Tensor("states_11/while/TensorArrayReadV3:0", shape=(?, 28), dtype=float32)
Wx <tf.Variable 'rnn_weights_11/W_x/Variable:0' shape=(28, 128) dtype=float32_ref>
b_rnn <tf.Variable 'rnn_weights_11/Bias/Variable:0' shape=(128,) dtype=float32_ref>
hh Tensor("states_11/while/MatMul_2:0", shape=(2, 128), dtype=float32)
xx Tensor("states_11/while/MatMul_3:0", shape=(?, 128), dtype=float32)
multi_1 Tensor("states_11/while/add_3:0", shape=(2, 128), dtype=float32)


In [46]:
print(_inputs)
print(processed_input)
print(initial_hidden)
print(all_hidden_states)

Tensor("inputs_1:0", shape=(?, 28, 28), dtype=float32)
Tensor("transpose_11:0", shape=(28, ?, 28), dtype=float32)
Tensor("zeros_1:0", shape=(128, 128), dtype=float32)
Tensor("states_1/TensorArrayStack/TensorArrayGatherV3:0", shape=(28, 128, 128), dtype=float32)


In [96]:
#%matplotlib inline
#import matplotlib
#import matplotlib.pyplot as plt
#batch_size = 2
#batch_x, batch_y = mnist.train.next_batch(batch_size)

#some_digit = batch_x[0]
#some_digit_image = some_digit.reshape(28,28)
#plt.imshow(some_digit_image, cmap = matplotlib.cm.binary, interpolation="nearest")
#plt.axis("off")
#plt.show()
#some_digit = batch_x[1]
#some_digit_image = some_digit.reshape(28,28)
#plt.imshow(some_digit_image, cmap = matplotlib.cm.binary, interpolation="nearest")
#plt.axis("off")
#plt.show()

In [122]:
batch_size=2
sess = tf.Session()
#batch_x, batch_y = mnist.train.next_batch(batch_size)

batch_x = batch_x.reshape((batch_size, time_steps, element_size))
out_inputs, out_y, out_processed_input = sess.run([_inputs, y, processed_input],feed_dict={_inputs: batch_x, y: batch_y})

In [100]:
out_processed_input.shape

(28, 2, 28)

In [111]:
a = np.array([[0,1,2,3],[4,5,6,7]])
b = np.array([0,1,2,3])
c = a + b
c

array([[ 0,  2,  4,  6],
       [ 4,  6,  8, 10]])

In [ ]:
# Weights for output layers
with tf.name_scope('linear_layer_weights') as scope:
    with tf.name_scope("W_linear"):
        Wl = tf.Variable(tf.truncated_normal([hidden_layer_size, num_classes],
                                             mean=0, stddev=.01))
        variable_summaries(Wl)
    with tf.name_scope("Bias_linear"):
        bl = tf.Variable(tf.truncated_normal([num_classes],
                                             mean=0, stddev=.01))
        variable_summaries(bl)


In [ ]:
with tf.name_scope('linear_layer_weights') as scope:
    # Iterate across time, apply linear layer to all RNN outputs
    all_outputs = tf.map_fn(get_linear_layer, all_hidden_states)
    # Get Last output -- h_28
    output = all_outputs[-1]
    tf.summary.histogram('outputs', output)

with tf.name_scope('cross_entropy'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=y))
    tf.summary.scalar('cross_entropy', cross_entropy)

with tf.name_scope('train'):
    # Using RMSPropOptimizer
    train_step = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cross_entropy)

with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(output, 1))

    accuracy = (tf.reduce_mean(tf.cast(correct_prediction, tf.float32)))*100
    tf.summary.scalar('accuracy', accuracy)

# Merge all the summaries
merged = tf.summary.merge_all()


# Get a small test set
test_data = mnist.test.images[:batch_size].reshape((-1, time_steps, element_size))
test_label = mnist.test.labels[:batch_size]

In [ ]:
with tf.Session() as sess:
    # Write summaries to LOG_DIR -- used by TensorBoard
    train_writer = tf.summary.FileWriter(LOG_DIR + '/train',
                                         graph=tf.get_default_graph())
    test_writer = tf.summary.FileWriter(LOG_DIR + '/test',
                                        graph=tf.get_default_graph())

    sess.run(tf.global_variables_initializer())

    for i in range(10000):

            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Reshape data to get 28 sequences of 28 pixels
            batch_x = batch_x.reshape((batch_size, time_steps, element_size))
            summary, _ = sess.run([merged, train_step],
                                  feed_dict={_inputs: batch_x, y: batch_y})
            # Add to summaries
            train_writer.add_summary(summary, i)

            if i % 1000 == 0:
                acc, loss, = sess.run([accuracy, cross_entropy],
                                      feed_dict={_inputs: batch_x,
                                                 y: batch_y})
                print("Iter " + str(i) + ", Minibatch Loss= " +
                      "{:.6f}".format(loss) + ", Training Accuracy= " +
                      "{:.5f}".format(acc))
            if i % 100 == 0:
                # Calculate accuracy for 128 mnist test images and
                # add to summaries
                summary, acc = sess.run([merged, accuracy],
                                        feed_dict={_inputs: test_data,
                                                   y: test_label})
                test_writer.add_summary(summary, i)

    test_acc = sess.run(accuracy, feed_dict={_inputs: test_data,
                                             y: test_label})
    print("Test Accuracy:", test_acc)


In [33]:
x = tf.Variable([[[1,2,3],[4,5,6]],
                [[7,8,9],[10,11,12]],
                 [[13,14,15],[16,17,18]]
                ], name='x')
model = tf.initialize_all_variables()
with tf.Session() as session:
    # Array 위치가 축 (x,y,z), 거기에 입력하는 숫자가 바꾸고 싶은 차원
    x = tf.transpose(x, perm=[1, 0, 2])
    session.run(model)
    print(session.run(x))


[[[ 1  2  3]
  [ 7  8  9]
  [13 14 15]]

 [[ 4  5  6]
  [10 11 12]
  [16 17 18]]]


In [27]:
result

array([[[ 1,  4],
        [ 2,  5],
        [ 3,  6]],

       [[ 7, 10],
        [ 8, 11],
        [ 9, 12]],

       [[13, 16],
        [14, 17],
        [15, 18]]], dtype=int32)